# Discussion Questions

1.
    2 particles = 1 pairwise particle-particle interactions \
    3 particles = 3 pairwise particle-particle interactions \
    4 particles = 6 pairwise particle-particle interactions \
    5 particles = 10 pairwise particle-particle interactions\
    10 particles = 45 pairwise particle-particle interactions\
    100 particles = 4950 pairwise particle-particle interactions\

    The original calculations without a cutoff was about -3487.45, whereas the calculations with a cutoff of a distance 3 resulted in -3582.23. This is approximately a 2.6% error and we think this would be a tolerable cutoff because it is less than 5%.

2.  
    Using a cutoff helps reduce the amount of calculation processing, which can help reduce computer memory usage and allow for more particles to be considered. As said in lecture, we are only at a point where a factor of millions of particles can be processed with calculations like these, yet a single mole contains 6.022*10^23. It is very helpful for us to minimize possibly negligible processing when we can to maximize our resources.
    A drawback is that the cutoff distance could be too high, and our results can become highly inaccurate because of this.

3.
    1. The maximum distance (σ) = 5*sqrt(3) = ~8.66
        The furthest coordinates away a point can be with a box length of 10 is 5 units in each dimension.
        For example, (0, 0, 0) and (5, 5, 5) have a distance of ~8.66σ using the calculate distance formula
    2. The actual distance = 2σ

In [101]:
# Necessary functions from lecture

import math

def calculate_LJ(r_ij):
    """
    The Lennard Jones interaction between two particles.

    Parameters
    -----------
    r_ij : float
        The distance between the two particles in reduced units.

    Returns
    -------
    pairwise_energy : float
        The pairwise energy between two particles in reduced units.
    """
    r6_term = math.pow((1 / r_ij), 6)
    r12_term = math.pow(r6_term, 2)
    pairwise_energy = 4 * (r12_term - r6_term)
    return pairwise_energy


def calculate_distance(coord1, coord2):
    """
    Calculate the distance between two 3D coordinates.
   
    Parameters
    ----------
    coord1, coord2: list
        The atomic coordinates
    
    Returns
    -------
    distance: float
        The distance between the two points.
    """

    dx = coord2[0]-coord1[0]
    dy = coord2[1]-coord1[1]
    dz = coord2[2]-coord1[2]

    d = math.sqrt(math.pow(dx,2) + math.pow(dy,2) + math.pow(dz,2))
    
    return d



def calculate_total_energy(coordinates, box_length, cutoff):
    """
    Calculate the total Lennard Jones energy of a system of particles.

    Parameters
    ----------
    coordinates : list
        Nested list containing particle coordinates.
    cutoff : float
        The cut-off distance.

    Returns
    -------
    total_energy : float
        The total pairwise Lennard Jones energy of the system of particles.
    """

    total_energy = 0

    num_atoms = len(coordinates)

    for i in range(num_atoms):
        for j in range(i + 1, num_atoms):

            # modified_calculate_distance function located later in code
            dist_ij = modified_calculate_distance(
                coordinates[i], coordinates[j], box_length=box_length
            )

            if dist_ij < cutoff:
                interaction_energy = calculate_LJ(dist_ij)
                total_energy += interaction_energy

    return total_energy

def read_xyz(filepath):
    """
    Reads coordinates from an xyz file.
    
    Parameters
    ----------
    filepath : str
       The path to the xyz file to be processed.
       
    Returns
    -------
    atomic_coordinates : list
        A two dimensional list containing atomic coordinates
    """
    
    with open(filepath) as f:
        box_length = float(f.readline().split()[0])
        num_atoms = float(f.readline())
        coordinates = f.readlines()
    
    atomic_coordinates = []
    
    for atom in coordinates:
        split_atoms = atom.split()
        
        float_coords = []
        
        # We split this way to get rid of the atom label.
        for coord in split_atoms[1:]:
            float_coords.append(float(coord))
            
        atomic_coordinates.append(float_coords)
        
    
    return atomic_coordinates, box_length


In [102]:
config1_file = "../data/sample_config1.txt"

sample_coords, box_length = read_xyz(config1_file)

## Task 4

In [103]:
# Modified calculate_distance function to include periodic boundaries

def modified_calculate_distance(coord1, coord2, box_length=None):
    """
    Calculate the distance between two 3D coordinates in a space.
   
    Parameters
    ----------
    coord1, coord2: list
        The atomic coordinates
    box_length: integer
        The length of one side of the box for periodic boundaries
    
    Returns
    -------
    distance: float
        The distance between the two points.
    """

    dx = abs(coord2[0]-coord1[0])
    dy = abs(coord2[1]-coord1[1])
    dz = abs(coord2[2]-coord1[2])

    # box length 10
    # 0 and 8
    # 10 - 8 = 2
    # 10 - 6 = 4

    if dx > (box_length/2):
        dx = box_length - dx
    if dy > (box_length/2):
        dy = box_length - dy
    if dz > (box_length/2):
        dz = box_length - dz

    d = math.sqrt(math.pow(dx,2) + math.pow(dy,2) + math.pow(dz,2))
    
    return d

In [104]:
test_coords_1 = [0, 0, 0]

# Tests the x coordinate to see if a point 8 units away will be calculated as 2
test_coords_2 = [8, 0, 0]
test_energy = modified_calculate_distance(test_coords_1, test_coords_2, box_length)
assert test_energy == 2

# Tests the y coordinate to see if a point 8 units away will be calculated as 2
test_coords_3 = [0, 8, 0]
test_energy = modified_calculate_distance(test_coords_1, test_coords_3, box_length)
assert test_energy == 2

# Tests the z coordinate to see if a point 8 units away will be calculated as 2
test_coords_4 = [0, 0, 8]
test_energy = modified_calculate_distance(test_coords_1, test_coords_4, box_length)
assert test_energy == 2

# Tests the x coordinate to see if a point 3 units away will remain 3 units away
test_coords_5 = [3, 0, 0]
test_energy = modified_calculate_distance(test_coords_1, test_coords_5, box_length)
assert test_energy == 3

# Tests the y coordinate to see if a point 3 units away will remain 3 units away
test_coords_6 = [0, 3, 0]
test_energy = modified_calculate_distance(test_coords_1, test_coords_6, box_length)
assert test_energy == 3

# Tests the z coordinate to see if a point 3 units away will remain 3 units away
test_coords_7 = [0, 0, 3]
test_energy = modified_calculate_distance(test_coords_1, test_coords_7, box_length)
assert test_energy == 3



# Tests for negative values
# Tests the x coordinate to see if a point 8 units away will be calculated as 2
test_coords_8 = [-8, 0, 0]
test_energy = modified_calculate_distance(test_coords_1, test_coords_8, box_length)
assert test_energy == 2

# Tests the y coordinate to see if a point 8 units away will be calculated as 2
test_coords_9 = [0, -8, 0]
test_energy = modified_calculate_distance(test_coords_1, test_coords_9, box_length)
assert test_energy == 2

# Tests the z coordinate to see if a point 8 units away will be calculated as 2
test_coords_10 = [0, 0, -8]
test_energy = modified_calculate_distance(test_coords_1, test_coords_10, box_length)
assert test_energy == 2

# Tests the x coordinate to see if a point 3 units away will remain 3 units away
test_coords_11 = [-3, 0, 0]
test_energy = modified_calculate_distance(test_coords_1, test_coords_11, box_length)
assert test_energy == 3

# Tests the y coordinate to see if a point 3 units away will remain 3 units away
test_coords_12 = [0, -3, 0]
test_energy = modified_calculate_distance(test_coords_1, test_coords_12, box_length)
assert test_energy == 3

# Tests the z coordinate to see if a point 3 units away will remain 3 units away
test_coords_13 = [0, 0, -3]
test_energy = modified_calculate_distance(test_coords_1, test_coords_13, box_length)
assert test_energy == 3

In [105]:
sample_energy_2 = calculate_total_energy(sample_coords, box_length, 3)
print(sample_energy_2)

assert math.isclose(sample_energy_2, -4351.5, rel_tol = 0.1)

-4351.540194543858


## Reflection - Task 4 - Thomas Janas

My first step with this task was trying to understand what the purpose of the periodic boundaries are for. After thinking about the concepts from lecture and discussing with the group, I realized it just helps to simulate the particles in an open space, whereas the simulations would in reality occur in a limited box based on the min and max of the random coordinates.

For the task, I made a function called modified_distance_calculation and I went into the calculate_total_energy function and changed it to use this new distance calculation instead of the old one. After our solid discussion and reading through the task, I tried to conceptualize how these imaginary boxes would work mathematically. The point would simply be repositioned if any of its coordinates was more than box_length/2 and after looking at examples and and modifying the points myself, I came up with the equation (for x here) x = box_length - x if x was larger than box_length/2. I felt like this was rather simple, but conceptually it does fulfill the imaginary boxes we spoke about in class. I developed a bunch of asserts statements, which probably could be simplified into 1 point having each coordinate changed instead of only altering 1 coordinate for each assert.After that, we ended up getting the same numbers that the NIST website listed, which was very satisfying to see. I remember earlier today I was not sure how the periodic boundaries worked, and I was somewhat uncomfortable with the coding for cutoff, but it all makes sense to me now. Which is very nice.
